In [1]:
import pandas as pd
import tldextract as domain_extract
import sys

Update `filename` (Rmb to include csv)

In [2]:
# Read Excel File
filename = "20240205-CFO_com-83 Leads.csv"

In [3]:
df = pd.read_csv(filename)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '20240205-CFO_com-83 Leads.csv'

In [ ]:
# List of columns required

col_inc = ['First Name', 'Middle Name', 'Last Name', 'Job Title', 'Job Function', 'Management Level', 
           'Email Address', 'Email Domain', 'Direct Phone Number', 'LinkedIn Contact Profile URL', 
           'Person Street', 'Person City', 'Person State', 'Person Zip Code', 'Country', 
           'Company Name', 'Website', 'Revenue (in 000s USD)', 'Revenue Range (in USD)', 
           'Primary Industry', 'Primary Sub-Industry', 'All Industries', 'All Sub-Industries', 
           'LinkedIn Company Profile URL', 'Facebook Company Profile URL', 'Twitter Company Profile URL',
           'Company Street Address', 'Company City', 'Company State', 'Company Zip Code', 'Company Country', 'Full Address']

No of Columns to Include: 32


In [ ]:
# Subset the required column

df = df[col_inc]

len_extract_col = df.shape[1]

In [ ]:
# Job Role (Standardized) and Management Level Cleaning
job_role = input("What is the job role for this ZI Purchase?")

mgmt_level = {"1":"C-Level", "2":"VP-Level", "3":"Others"}

# default value for mgmt_input
mgmt_input = 0

while mgmt_input not in ["1","2", "3", "4"]: # 4 = Using Default Value
    mgmt_input = input("What is the management level? 1 for C-Level; 2 for VP-Level; 3 for Others; 4 for Default Value")

df['Job Role (Standardized)'] = job_role

if mgmt_input in ["1","2", "3"]:
    df['Management Level'] = mgmt_level[mgmt_input]

In [ ]:
# Domain Extraction
df['Company Domain'] = df['Website'].apply(lambda x: domain_extract.extract(x).domain + "." + domain_extract.extract(x).suffix)

In [ ]:
# Add New Column for ICP Check
df['Industry (Standardized)'] = ""
df['Lead Segments HS'] = ""
df['Industry Re-Segmentation'] = ""

In [ ]:
# Add New Column for Additional Info
df['Membership Note'] = ""
df['Source (Self-Define)'] = ""

In [14]:
# ICP Checking
def services_icp_check(dataset):
    
    # Add columns for validation
    dataset['Valid/Invalid'] = ""
    dataset['Invalid Reason'] = ""

    # ------------------------------------------------------------------------------------------

    # Contact Job Title Checking is done during the preview stage
    # Location ICP
    country_ICP = ['United States', 'Canada']

    def country_check (row):
        if row['Country'] not in country_ICP:
            return 'Contacts Not In US/CA'
        elif row['Company Country'] not in country_ICP:
            return 'Company Not In US/CA'
        else:
            return ""

    dataset['Invalid Reason'] = dataset.apply(country_check, axis=1)

    # ------------------------------------------------------------------------------------------

    # Company Revenue Check
    def revenue_check(row):
        if row['Invalid Reason'] != "":
            return row['Invalid Reason']
        elif row['Revenue (in 000s USD)'] < 1000000:
            return 'Invalid Revenue Range'
        else:
            return ""

    dataset['Invalid Reason'] = dataset.apply(revenue_check, axis=1)

    # ------------------------------------------------------------------------------------------

    # Assign Invalid
    def tag_invalid(row):
        if row['Invalid Reason'] != "":
            return "Invalid"
        else:
            return "Double Check for Job Title"
        
    dataset['Valid/Invalid'] = dataset.apply(tag_invalid, axis=1)

    return dataset


In [ ]:
# List of columns required

col_rearrange = ['First Name', 'Middle Name', 'Last Name', 'Job Title', 'Job Role (Standardized)', 'Job Function', 'Management Level', 
                 'Email Address', 'Email Domain', 'Direct Phone Number', 'LinkedIn Contact Profile URL', 
                 'Person Street', 'Person City', 'Person State', 'Person Zip Code', 'Country', 
                 'Company Name', 'Website', 'Company Domain', 'Revenue (in 000s USD)', 'Revenue Range (in USD)', 
                 'Primary Industry', 'Primary Sub-Industry', 'All Industries', 'All Sub-Industries', 'Industry (Standardized)', 'Lead Segments HS', 'Industry Re-Segmentation',
                 'LinkedIn Company Profile URL', 'Facebook Company Profile URL', 'Twitter Company Profile URL', 
                 'Company Street Address', 'Company City', 'Company State', 'Company Zip Code', 'Company Country', 'Full Address',
                 'Membership Note', 'Source (Self-Define)']

No of Columns for Final Output: 39


In [ ]:
# Rearrange column
df = df[col_rearrange]

In [15]:
output_filename = filename.strip(".csv") + "-Output.csv"

In [16]:
print(output_filename)

20240205-CFO_com-83 Lead-Output.csv


In [17]:
# Export
df.to_csv(output_filename, index= False)